In [1]:
import pandas as pd
import random

## 1.读取特征

In [2]:
feature = pd.read_csv('../processing/feature.csv')

In [3]:
Entry = feature['Entry'].values.tolist()
Gene_names = feature['Gene names'].values.tolist()
Gene_names = [str(i).split() for i in Gene_names]
Gene_name = [i[0] for i in Gene_names]
STRING = feature['Cross-reference (STRING)'].values.tolist()

In [4]:
Entry_name_id_map = dict(zip(feature['Entry name'],feature.index))
Entry_id_map = dict(zip(feature['Entry'],feature.index))
id_Entry_map = dict(zip(feature.index,feature['Entry']))
Entry_name_Entry_map = dict(zip(feature['Entry name'],feature['Entry']))
STRING_id_map = dict(zip(feature['Cross-reference (STRING)'],feature.index))
STRING_Entry_map = dict(zip(feature['Cross-reference (STRING)'],feature['Entry']))
STRING_Gene_name_map = dict(zip(feature['Cross-reference (STRING)'],Gene_name))
Entry_Entry_name = dict(zip(feature['Entry'],feature['Entry name']))
Entry_Genes_name = dict(zip(feature['Entry'],Gene_names))
Entry_Gene_name = dict(zip(feature['Entry'],Gene_name))

In [5]:
id_Gene_map = dict(zip(feature.index,Gene_name))

## 2.读取DUB-底物相互作用数据集

In [9]:
DSI = pd.read_csv('human_dsi_year.csv')
print(DSI.shape)

#删除含有模糊氨基酸编码的蛋白，例如'Q92813'(DIO2)
DSI = DSI[(DSI['SwissProt AC (DUB)'].isin(Entry))&(DSI['SwissProt AC (Substrate)'].isin(Entry))]
print(DSI.shape)

(868, 15)
(861, 15)


## 3.划分训练集和测试集

In [10]:
dsi_train = DSI[DSI['year'] == 'before 2018.1.1'] #559
dsi_test = DSI[DSI['year'] == 'after 2018.1.1'] #302

print(dsi_train.shape,dsi_test.shape)


(559, 15) (302, 15)


In [12]:
# DSI.to_csv('../../networks/gsp_dsi.csv')


## 4.编码 ID

In [13]:
def encoding_id(data):
    DUB = data['SwissProt AC (DUB)']
    substrate = data['SwissProt AC (Substrate)']
    DUB_ID = [Entry_id_map[i]  for i in DUB]
    substrate_ID = [Entry_id_map[i]  for i in substrate]
    dsi = [[DUB_ID[i],substrate_ID[i]] for i in range(len(DUB_ID))]
    return dsi

In [14]:
gsp_train_id  = encoding_id(dsi_train) #559
gsp_test_id = encoding_id(dsi_test) #302

In [15]:
gsp_id = encoding_id(DSI)  #861

## 5.输出训练集和测试集

In [37]:
def output_file(outputfile,data):
    with open('../../networks/'+outputfile,'w') as f:
        for row in data:
            print(row[0],'\t',row[1],file=f)

In [38]:
output_file('gsp_train.txt',gsp_train_id)
output_file('gsp_test.txt',gsp_test_id)
# output_file('gsp.txt',gsp_id)

## 6.string.txt构建ppi网络

In [18]:
string = pd.read_table('../string/ppi.txt',delimiter=" ")
string = string[['protein1','protein2','combined_score']]
print(string.shape)

(11759454, 3)


In [19]:
string = string[string['protein1'].isin(STRING)]
string = string[string['protein2'].isin(STRING)]
print(string.shape)

(11065260, 3)


In [20]:
string['protein1_Entry'] = string['protein1'].apply(lambda x:STRING_Entry_map[x])
string['protein2_Entry'] = string['protein2'].apply(lambda x:STRING_Entry_map[x])
string['protein1_Gene'] = string['protein1'].apply(lambda x:STRING_Gene_name_map[x])
string['protein2_Gene'] = string['protein2'].apply(lambda x:STRING_Gene_name_map[x])

In [22]:
#STRING的PPI在Uniprot数据中有对应名字的PPI
string.to_csv('../string/ppi_entry.txt')

## 7.从选取金标准阴性数据集

In [23]:
DUBs = set(DSI['SwissProt AC (DUB)'].values.tolist())
substrates = set(DSI['SwissProt AC (Substrate)'].values.tolist())
print(len(DUBs),len(substrates)) #DUB和底物种类数

86 486


#### 7.1 从STRING数据库的PPI网络中选取与DUB发生相互作用但不是底物的ppi作为阴性集

In [24]:
#删除已知DSI中包含的底物
dub = list(set([k for line in gsp_id for k,v in STRING_id_map.items() if v ==line[0]]))
sub = list(set([k for line in gsp_id for k,v in STRING_id_map.items() if v ==line[1]]))
sub_not = list(set(STRING)-set(sub))
print(len(dub),len(sub),len(sub_not))

85 484 18044


In [25]:
string_dub = string[string['protein1'].isin(dub)]
print(string_dub.shape)

(58139, 7)


In [26]:
string_dub['DUB'] = string_dub['protein1'].apply(lambda x:STRING_Entry_map[x])
string_dub['substrate'] = string_dub['protein2'].apply(lambda x:STRING_Entry_map[x])
string_dub['DUB Gene name'] = string_dub['protein1'].apply(lambda x:STRING_Gene_name_map[x])
string_dub['substrate Gene name'] = string_dub['protein2'].apply(lambda x:STRING_Gene_name_map[x])
string_dub['DUB_id'] = string_dub['protein1'].apply(lambda x:STRING_id_map[x])
string_dub['substrate_id'] = string_dub['protein2'].apply(lambda x:STRING_id_map[x])

<ipython-input-26-a1f6aff759c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  string_dub['DUB'] = string_dub['protein1'].apply(lambda x:STRING_Entry_map[x])
<ipython-input-26-a1f6aff759c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  string_dub['substrate'] = string_dub['protein2'].apply(lambda x:STRING_Entry_map[x])
<ipython-input-26-a1f6aff759c3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [29]:
string_dub.to_csv("../string/ppi_DUB.txt") #STRING数据库中包含DUB的PPI

In [30]:
string_dub = string_dub[string_dub['combined_score']>300] #得分大于300,
# string_dub.to_csv("../processing/string_dsi_socre_over_300.csv")

In [31]:
string_select = string_dub[string_dub['protein2'].isin(sub_not)]

In [32]:
gsn_all = []
for row in string_select.values:
    p1 = STRING_id_map[row[0]]
    p2 = STRING_id_map[row[1]]
    ppi = [p1,p2]
    if ppi not in gsp_train_id:
        gsn_all.append(ppi)

#### 7.2 输出金标准阴性ppi

In [33]:
random.seed(2)
gsn_id = random.sample(gsn_all,len(gsp_id))
gsn_train_id = random.sample(gsn_id,len(gsp_train_id))
gsn_test_id = [i for i in gsn_id if i not in gsn_train_id]

In [34]:
def id_name_map(gsn_id):
    gsn_filter = []
    for line in gsn_id:
        p1 = id_Entry_map[line[0]]
        p2 = id_Entry_map[line[1]]
        gsn_filter.append([p1,p2])
    return gsn_filter

In [35]:
gsn_filter = id_name_map(gsn_id)
gsn_train_filter = id_name_map(gsn_train_id)
gsn_test_filter = id_name_map(gsn_test_id)


In [32]:
#阴性数据集的名称
# output_file('gsn_filter.txt',gsn_filter)
# output_file('gsn_train_filter.txt',gsn_train_filter)
# output_file('gsn_test_filter.txt',gsn_test_filter)


In [39]:
# output_file('gsn.txt',gsn_id)
output_file('gsn_train.txt',gsn_train_id)
output_file('gsn_test.txt',gsn_test_id)


#### 输出ppi数据集

In [28]:
# 从string_ppi中删除金标准阳性ppi和选择的阴性ppi
with open('networks/ppi.txt','w') as f:
    for row in string.values:
        p1 = STRING_id_map[row[0]]
        p2 = STRING_id_map[row[1]]
        ppi = [p1,p2]
        if (ppi not in gsp_id) and (ppi not in gsn_id):
            print(p1,'\t',p2,file=f)

FileNotFoundError: [Errno 2] No such file or directory: '../networks/ppi.txt'

## 8.从阴性数据集中选取随机对照

In [62]:
DSIs_pre = pd.read_csv("../output/Validation of bioinformatics methods/nogsd_string_predict_dsi.csv")
print(DSIs_pre.shape[0])
DSIs_pre = DSIs_pre[DSIs_pre['score']>0.87]
print(DSIs_pre.shape[0])

57573
5341


In [63]:
def id_name_map2(gsn_id):
    data = pd.DataFrame(columns=['DUB_Genes_name', 'substrate_Genes_name'])
    p1_list = []
    p2_list = []
    p1_ = []
    p2_ = []
    for line in gsn_id:
        p1 = id_Gene_map[line[0]]
        p2 = id_Gene_map[line[1]]
        p1_list.append(p1)
        p2_list.append(p2)
        p1_.append(line[0])
        p2_.append(line[1])
    data['DUB_Genes_name'] = p1_list
    data['substrate_Genes_name'] = p2_list
    data['DUB'] = p1_
    data['substrate'] = p2_
    return data

In [64]:
gsn_filter_ = id_name_map2(gsn_all)
print(gsn_filter_.shape)
gsn_filter_ = gsn_filter_[gsn_filter_['DUB_Genes_name'].isin(set(DSI['Gene Symbol (DUB)'].values.tolist()))]
print(gsn_filter_.shape)

(14613, 4)
(14591, 4)


In [41]:
for i in range(1000):
    gsn_filter_random_compared = gsn_filter_.sample(n = DSIs_pre.shape[0])
    gsn_filter_random_compared.to_csv('../../circle/random/gsn_random_compared' + str(i+1)+'.csv')

